In [1]:
from music21 import converter,instrument # or import *

In [3]:
file = converter.parse('JeSuisRobert.mid')
components = []
for element in file.recurse():
    components.append(element)
print(components)

[<music21.stream.Part 0x7f52c8412050>, <music21.instrument.Vocalist 'Voice'>, <music21.tempo.MetronomeMark Quarter=100.0>, <music21.key.Key of F major>, <music21.meter.TimeSignature 1/4>, <music21.note.Rest rest>, <music21.note.Note G>, <music21.meter.TimeSignature 2/2>, <music21.note.Note D>, <music21.note.Note F>, <music21.note.Note C>, <music21.note.Note C>, <music21.note.Note B->, <music21.note.Note C>, <music21.note.Note D>, <music21.note.Note B->, <music21.note.Note A>, <music21.note.Note B->, <music21.note.Note C>, <music21.note.Note D>, <music21.note.Note G>, <music21.note.Note B->, <music21.note.Note A>, <music21.note.Note A>, <music21.meter.TimeSignature 1/2>, <music21.note.Note G>, <music21.note.Rest rest>, <music21.note.Note G>, <music21.meter.TimeSignature 2/2>, <music21.note.Note G>, <music21.note.Note A>, <music21.note.Note B->, <music21.note.Note C>, <music21.note.Note D>, <music21.note.Note D>, <music21.note.Note E>, <music21.note.Note D>, <music21.note.Note C>, <music